#### Auth for Spotify

In [ ]:
import pandas as pd
from auth import auth_with_credentials
from dotenv import load_dotenv
import os

load_dotenv()
SPOTIFY_ID=os.getenv('SPOTIFY_ID')
SPOTIFY_SECRET=os.getenv('SPOTIFY_SECRET')
redirect_uri = 'https://spotify.homeclean.site/callback'

In [ ]:
sp = auth_with_credentials(client_id=SPOTIFY_ID, client_secret=SPOTIFY_SECRET)

In [ ]:
from spotipy.oauth2 import SpotifyOAuth
import spotipy
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=SPOTIFY_ID,
        client_secret=SPOTIFY_SECRET,
        redirect_uri="https://spotify.homeclean.site/callback",
        scope = (
            "user-read-email "
            "user-library-read "
            "user-read-playback-state "
            "user-read-currently-playing "
            "user-modify-playback-state "
            "app-remote-control "
            "playlist-read-collaborative "
            "playlist-read-private"
        ),
        cache_handler=None
    )
)


In [ ]:
import spotipy
access_token='BQAXn3dt3V5VbfhV94uWv8H9IYajUQ4aFf2wSeNqpZbOR18fYs7rMdggy6x5HT1ZLKkVE9Spi-sB9h1ktcJum6hpa-JjfCJ8uJo_UCdBmDfGRoK98kCibgoFufhjQ7uzsbdMVi5znnNB860FRh51DzVXOPaps-nXSbjMs-Q3JEoFhVgwFYX94f6gLYCMaoFeuGZjWhnNNH-1vGFwC0g9dI0vsp8Y2oavOLM1EnZObbGBSJKyBbbogBW41hWYUMZmRZg8'
sp = spotipy.Spotify(auth=access_token)
print(sp.me())


In [ ]:
print(sp.current_playback())

In [ ]:
current_user_all_playlist = sp.current_user_playlists()

In [ ]:
import json
current_user_all_playlist['items'][0]

In [ ]:
play_list_name = current_user_all_playlist['items'][0]['name']
playlist_id = current_user_all_playlist['items'][0]['id']
owner_name = current_user_all_playlist['items'][0]['owner']['display_name']
owner_uri = current_user_all_playlist['items'][0]['owner']['uri']
is_public_playlist = current_user_all_playlist['items'][0]['public']
total = current_user_all_playlist['items'][0]['tracks']['total']
playlist_uri = current_user_all_playlist['items'][0]['uri']

In [ ]:
list_1 = []
list_1.append({
    'play_list_name':play_list_name,
    'playlist_id':play_list_name,
    'owner_name':owner_name,
    'owner_uri':owner_uri,
    'is_public_playlist':is_public_playlist,
    'total':total,
    'playlist_uri':playlist_uri
})

list_1

In [ ]:
list_1

In [ ]:
playlist_name = []
playlist_id = []
owner_name = []
owner_uri = []
is_public_playlist = []
total = []
playlist_uri = []

for i in range(0, 6):
    playlist_name.append(str(current_user_all_playlist['items'][i].get('name')))
    playlist_id.append(str(current_user_all_playlist['items'][i].get('id')))
    owner_name.append(str(current_user_all_playlist['items'][i]['owner'].get('display_name')))
    owner_uri.append(str(current_user_all_playlist['items'][i]['owner'].get('uri')))
    is_public_playlist.append(current_user_all_playlist['items'][i].get('public'))
    total.append(current_user_all_playlist['items'][i]['tracks'].get('total'))
    playlist_uri.append(str(current_user_all_playlist['items'][i].get('uri')))

df = pd.DataFrame({
    'playlist_name': playlist_name,
    'playlist_id': playlist_id,
    'owner_name': owner_name,
    'owner_uri': owner_uri,
    'is_public_playlist': is_public_playlist,
    'total': total,
    'playlist_uri': playlist_uri
})


In [ ]:
df = pd.json_normalize(current_user_all_playlist['items'])

In [ ]:
df.to_csv('raw_data/playlist.csv', index=False)

In [ ]:
import json

items = json.loads(json.dumps(current_user_all_playlist["items"]))

items

In [ ]:
playlist_name = []
playlist_id = []
owner_name = []
owner_uri = []
is_public_playlist = []
total = []
playlist_uri = []

for pl in items:
    playlist_name.append(pl["name"] )
    playlist_id.append(pl["id"] )
    owner_name.append(pl["owner"]["display_name"] )
    owner_uri.append(pl["owner"]["uri"].split(':')[-1] )
    is_public_playlist.append(pl["public"] )
    total.append(pl["tracks"]["total"] )
    playlist_uri.append(pl['uri'].split(':')[-1])

In [ ]:
playlist_id = df['playlist_uri'][0].split(':')[-1]

In [ ]:
playlist_id

In [ ]:
data = sp.playlist(playlist_id)

In [ ]:
len(data['tracks']['items'])

In [ ]:
sp.audio_analysis(track_id='5J90ah0ppUSev1uahqQiN6')

In [ ]:
sp.search(q='Vũ.')

#### Get raw data of some track

##### Get track infomation from Playlists on Spotify

In [30]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from dotenv import load_dotenv
import os

load_dotenv()
SPOTIFY_ID=os.getenv('SPOTIFY_ID')
SPOTIFY_SECRET=os.getenv('SPOTIFY_SECRET')

In [3]:
import requests
base_url="https://acousticbrainz.org"

In [31]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_ID,
    client_secret=SPOTIFY_SECRET
))

track_id = "2P4OICZRVAQcYAV2JReRfj"  # ví dụ
track_info = sp.track(track_id)

track_name = track_info["name"]
artist_name = track_info["artists"][0]["name"]
print("Track:", track_name, "| Artist:", artist_name)

Track: Waiting For Love | Artist: Avicii


In [7]:
mb_search_url = "https://musicbrainz.org/ws/2/recording/"
params = {
    "query": f'track:"{track_name}" AND artist:"{artist_name}"',
    "fmt": "json"
}
resp = requests.get(mb_search_url, params=params, headers={"User-Agent": "Spotify2AcousticBrainz/1.0"})
mb_data = resp.json()

if "recordings" in mb_data and len(mb_data["recordings"]) > 0:
    mbid = mb_data["recordings"][0]["id"]
    print("MusicBrainz ID:", mbid)
else:
    print("❌ Không tìm thấy MBID")
    exit()

MusicBrainz ID: f8675903-5235-4e5c-a9be-89dd87341f3e


In [11]:
raw_data = sp.playlist_items(playlist_id='5oYN7C2h3aRI9OGNpXsd0F')

In [12]:
items = raw_data['items']

In [ ]:
track_id = items[0]['track']['id']
track_name = items[0]['track']['name']
artist_name = items[0]['track']['artists'][0]['name']
popularity = items[0]['track']['popularity']
duration_ms = items[0]['track']['duration_ms']

'Avicii'

In [16]:
tracks_id = []
tracks_name = []
artist_name = []
popularity = []
duration_ms = []

for i, track in enumerate(items):
    tracks_id.append(track['track']['id'])
    tracks_name.append(track['track']['name'])
    artist_name.append(track['track']['artists'][0]['name'])
    popularity.append(track['track']['popularity'])
    duration_ms.append(track['track']['duration_ms'])


In [29]:
import pandas as pd
from datetime import datetime, timedelta


df_tracks = pd.DataFrame({
    "track_id": tracks_id,
    "track_name": tracks_name,
    "artist_name": artist_name,
    "popularity": popularity,
    "duration_ms": duration_ms
})

df_tracks.to_csv(
    f"raw_data/tracks_name_{datetime.today().strftime('%Y%m%d')}.csv",
    index=False
)

In [20]:
import requests
from bs4 import BeautifulSoup
data = requests.get('https://www.pvoil.com.vn/tin-gia-xang-dau')

if data.status_code == 200:
    print('ok')

ok


In [ ]:
data.content
soup = BeautifulSoup(data.content, 'lxml')

In [23]:
import pandas as pd
from datetime import datetime
from io import StringIO

table = soup.find('table')

# Bọc HTML string bằng StringIO
df = pd.read_html(StringIO(str(table)))[0]

# Giữ 2 cột cần thiết
df = df.iloc[:, [1, 3]]
df.columns = ["Categories", "Price(VND/l)"]
df['updated_date'] = datetime.today()
df["Price(VND/l)"] = (
    df["Price(VND/l)"]
    .astype(str)
    .str.replace("đ", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.strip()
    .astype(int)
)
df.to_csv('raw_data/fuel_price.csv', index=False, mode='a')

##### Get audio feature

In [33]:
df_1 = pd.read_csv('raw_data/tracks_name_20250904.csv')

In [ ]:
import requests
base_url='https://musicbrainz.org/ws/2/recording'